## Neural network based on backward automatic differentiation

### Iris dataset

Kinds of irises:
    * setosa     = [1.0  0.0  0.0]
    * versicolor = [0.0  1.0  0.0]
    * virginica  = [0.0  0.0  1.0]

#### Import packages

In [1]:
include("data_preparing.jl");

####  Data preparing

In [2]:
iris_dataset = prepare_iris();
train_set, test_set = split_dataset(iris_dataset, 0.8);
test_set, test_expected_values = adjust_dataset(test_set);
train_set, train_expected_values = adjust_dataset(train_set);

### Neural network

#### Import packages

In [3]:
using LinearAlgebra
include("automatic_differentiation.jl");

#### Hyperparameters definiton

In [4]:
α = 0.1 # learning rate
epochs = 10;

#### Design architecture

In [5]:
x = Variable(train_set[1,:], name="x")
Wh = Variable(rand(10, 4), name="Wh")
Wo = Variable(rand(3, 10), name="Wo")
y = Variable(train_expected_values[1,:], name="y");

In [6]:
function dense(w, b, x, activation) return activation.(w * x .+ b) end
function dense(w, x, activation) return activation.(w * x) end
function dense(w, x) return w * x end

function mean_squared_loss(y, ŷ)
    return Constant(1.0/3.0) .* sum.((y .- ŷ) .^ Constant(2))
end

function net(x, Wh, Wo, y)
    x̂ = dense(Wh, x, σ)
    x̂.name = "x̂"
    ŷ = dense(Wo, x̂, softmax)
    ŷ.name = "ŷ"
    E = mean_squared_loss(y, ŷ)
    E.name = "total_loss"
    return topological_sort(E)
end    

net (generic function with 1 method)

In [7]:
function dnet(E)
    println("total_loss\t= ", forward!(E))
    println("result\t\t= ", E[9].output)
    println()
    backward!(E)
    tuple(E[4].gradient, E[3].gradient)
end

dnet (generic function with 1 method)

In [8]:
E = net(x, Wh, Wo, y)
for i=1:epochs
    dWh, dWo = dnet(E)
    Wh.output -= α * dWh
    Wo.output -= α * dWo
end

total_loss	= 0.31426169420515576
result		= [0.6108334471887078, 0.25691397641488395, 0.13225257639640825]

total_loss	= 0.26032318859360765
result		= [0.5431608593664752, 0.3170596384692189, 0.13977950216430582]

total_loss	= 0.20857705132176213
result		= [0.4741515409025572, 0.3831205774106184, 0.1427278816868244]

total_loss	= 0.16360844364926877
result		= [0.40962818993756533, 0.4494024106590039, 0.1409693994034308]

total_loss	= 0.12770514951045486
result		= [0.3537204194994428, 0.5105392098164989, 0.1357403706840582]

total_loss	= 0.10058825916807748
result		= [0.3077270899764373, 0.5635405999057811, 0.1287323101177816]

total_loss	= 0.08060324534063783
result		= [0.2708426535645073, 0.6078919755324963, 0.12126537090299631]

total_loss	= 0.06589552131853608
result		= [0.2414262397007708, 0.6444926935398018, 0.1140810667594274]

total_loss	= 0.054942947832377645
result		= [0.21781882855125703, 0.6746867036081187, 0.10749446784062433]

total_loss	= 0.04664040479415617
result		= [0.1